2021.08.09 OIE疫情資料爬蟲

In [13]:
import pandas as pd
import requests
from datetime import datetime, timedelta

- 目標網站:
    - https://wahis.oie.int/pi/getLatestReports (最新版的JS，GET)
    - https://wahis.oie.int/pi/getReportList (詳細版的JS，POST)

In [14]:
# url =  "https://wahis.oie.int/pi/getLatestReports"
url_all = 'https://wahis.oie.int/pi/getReportList'

# 最新版

In [4]:
# df = pd.read_json(url)
# df

,country,disease,date,diseaseType
0,Russia,African swine fever virus (Inf. with),2021-08-06 15:38:58.316000+00:00,None
1,Russia,African swine fever virus (Inf. with),2021-08-06 14:57:21.687000+00:00,None
2,Togo,Highly pathogenic avian influenza (poultry),2021-08-02 17:29:45.100000+00:00,H5N1
3,Russia,African swine fever virus (Inf. with),2021-07-30 13:09:07.517000+00:00,None
4,Russia,African swine fever virus (Inf. with),2021-07-30 13:07:23.263000+00:00,None
5,Dominican (Rep),African swine fever virus (Inf. with),2021-07-29 19:36:52.879000+00:00,None
6,Kenya,Rift Valley fever virus (Inf. with),2021-07-28 14:49:17.388000+00:00,None
7,Russia,Newcastle disease virus (Inf. with),2021-07-28 14:43:55.868000+00:00,None


# 詳細版

In [15]:
rq_data = {
  "pageNumber":1,
  "pageSize":500,
  "searchText":"",
  "sortColName":"",
  "sortColOrder":"ASC",
  "reportFilters":{"reportDate":{"startDate":(datetime.today() + timedelta(days=-14)).strftime("%Y-%m-%d"),
                                 "endDate":datetime.today().strftime("%Y-%m-%d")}},
  "languageChanged":"false"
}

In [16]:
rq_data # 近2週

{'pageNumber': 1,
 'pageSize': 500,
 'searchText': '',
 'sortColName': '',
 'sortColOrder': 'ASC',
 'reportFilters': {'reportDate': {'startDate': '2021-07-26',
   'endDate': '2021-08-09'}},
 'languageChanged': 'false'}

In [17]:
r = requests.post(url_all, json=rq_data) # POST內容有nested json 要用參數json= 而非 data=
df_all = pd.DataFrame(r.json()["homePageDto"])
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   country              132 non-null    object
 1   reportId             132 non-null    int64 
 2   status               132 non-null    object
 3   diseases             132 non-null    object
 4   reason               132 non-null    object
 5   eventDate            132 non-null    object
 6   reportDate           132 non-null    object
 7   reportType           132 non-null    object
 8   reportInfoId         132 non-null    int64 
 9   eventStatus          132 non-null    object
 10  flagTranslated       132 non-null    bool  
 11  region               132 non-null    object
 12  reportNumber         132 non-null    int64 
 13  reportHistoryType    132 non-null    object
 14  eventIdOIEReference  132 non-null    object
 15  diseaseType          60 non-null     object
 16  reportRe

In [18]:
df_all.head()

,country,reportId,status,diseases,reason,eventDate,reportDate,reportType,reportInfoId,eventStatus,flagTranslated,region,reportNumber,reportHistoryType,eventIdOIEReference,diseaseType,reportReference
0,Laos,150861,Validated,Lumpy skin disease virus (Inf. with),First occurrence in the country,2021-05-22,2021-07-26,FUR,36628,On-going,True,Asia,0,Follow-up report2,evt_3788,NaN,NaN
1,Moldova,150863,Validated,African swine fever virus (Inf. with),First occurrence in a zone or a compartment,2020-04-07,2021-07-26,FUR,37043,On-going,True,Europe,0,Follow-up report59,33944,NaN,NaN
2,Cuba,150866,Validated,Rabbit haemorrhagic disease,New strain in the country,2021-06-14,2021-07-26,FUR,36639,On-going,True,Americas,0,Follow-up report5,evt_3774,RHDV2,NaN
3,Japan,150860,Validated,Equine piroplasmosis,First occurrence in the country,2021-07-24,2021-07-26,IN,36810,On-going,True,Asia,0,Immediate notification,evt_3836,NaN,NaN
4,Sweden,150868,Validated,Highly pathogenic influenza A viruses (Inf. wi...,Recurrence,2020-10-30,2021-07-26,FUR,36725,On-going,True,Europe,0,Follow-up report34,36791,H5N8,NaN


## 挑選重點疫情

- 挑選邏輯:
    - diseases 文字敘述中有 Highly pathogenic
    - diseaseType 有值(有找到"H")
    - status 為 Validated


In [19]:
mask = (df_all["diseases"].str.contains('Highly pathogenic', regex=False).fillna(False)) \
            & (df_all["diseaseType"].str.contains('H', regex=False).fillna(False)) \
            & (df_all["status"].isin(["Validated"]))
df_all_pic = df_all.loc[mask,:]
df_all_pic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43 entries, 4 to 116
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   country              43 non-null     object
 1   reportId             43 non-null     int64 
 2   status               43 non-null     object
 3   diseases             43 non-null     object
 4   reason               43 non-null     object
 5   eventDate            43 non-null     object
 6   reportDate           43 non-null     object
 7   reportType           43 non-null     object
 8   reportInfoId         43 non-null     int64 
 9   eventStatus          43 non-null     object
 10  flagTranslated       43 non-null     bool  
 11  region               43 non-null     object
 12  reportNumber         43 non-null     int64 
 13  reportHistoryType    43 non-null     object
 14  eventIdOIEReference  43 non-null     object
 15  diseaseType          43 non-null     object
 16  reportRef

## 疫情統整

In [20]:
# 以 reportDate、country 為 group， 統計疫情數量
gp_cols = ["reportDate","country","diseaseType"]
df_report = df_all_pic[gp_cols+["reportId"]].groupby(gp_cols, as_index=False).count()
df_report.rename(columns={"reportId":"report_count"}, inplace=True)
df_report.sort_values(by=["reportDate"], ascending=[0], inplace=True)
df_report.reset_index(drop=True, inplace=True)
df_report

,reportDate,country,diseaseType,report_count
0,2021-08-06,Chinese Taipei,H5N5,1
1,2021-08-05,Togo,H5N1,1
2,2021-08-05,France,H5N8,1
3,2021-08-04,Lithuania,H7N7,1
4,2021-08-04,Lithuania,H5N8,2
5,2021-08-04,France,H5N3,1
6,2021-08-04,France,H5N1,1
7,2021-08-03,Ghana,H5N8,1
8,2021-08-03,Finland,H5N1,1
9,2021-08-03,Finland,H5N8,1


## 欄位名稱對齊資料庫

In [21]:
# OIE 原始資料 範例
# {
#     "RelatedReports": "NULL",
#     "CausalAgent": "HPAI",
#     "DateOfConfirmationOfTheEvent": "14/02/2020",
#     "ManifestationOfDisease": "Clinical disease",
#     "DateEventResolved": "NULL",
#     "ReasonForNotification": "Recurrence of a listed disease",
#     "ThisEventPertainsTo": "A Defined Zone Within The Country",
#     "outbreaks": "2",
#     "NatureOfDiagnosis": "Clinical, Laboratory (advanced)",
#     "DateOfPreviousOccurrence": "04/06/2019",
#     "DateSubmittedToOie": "2020-02-17",
#     "Serotype": "H5N1",
#     "ReportType": "Immediate notification",
#     "lat": null,
#     "_id": "33305",
#     "description": "Sourced from OIE website.",
#     "DateOfStartOfTheEvent": "11/02/2020",
#     "area": "Vietnam",
#     "lon": null,
#     "ReportDate": "17/02/2020",
#     "url": "https://www.oie.int/wahis_2/public/wahid.php/Reviewreport/Review?page_refer=MapFullEventReport&reportid=33305"
# }

In [22]:
df_report.head(1).to_dict()

{'reportDate': {0: '2021-08-06'},
 'country': {0: 'Chinese Taipei'},
 'diseaseType': {0: 'H5N5'},
 'report_count': {0: 1}}

In [23]:
# 重新命名
col_rename_dict = {
 'reportDate': "ReportDate",
 'country': "area",
 'diseaseType': "Serotype",
 'report_count': "outbreaks"
}
df_report.rename(columns=col_rename_dict, inplace=True)

# 還原欄位(舊資料有，新資料沒有，但對功能沒有意義)
rebuildColsLst = ["RelatedReports", "DateOfConfirmationOfTheEvent", "ManifestationOfDisease", "DateEventResolved",
                 "ReasonForNotification", "ThisEventPertainsTo", "NatureOfDiagnosis", "DateOfPreviousOccurrence",
                 "DateSubmittedToOie", "ReportType", "lat", "description", "DateOfStartOfTheEvent","lon"]
for _c in rebuildColsLst:
    df_report.loc[:, _c] = None


# 創建欄位(舊資料有，新資料沒有，對功能有意義)
df_report.loc[:, "_id"] = df_report.loc[:, "ReportDate"] + "_" + df_report.loc[:, "area"]
df_report.loc[:, "CausalAgent"] = "HPAI"
df_report.loc[:, "url"] = "https://wahis.oie.int/#/events?viewAll=true"

# 對齊欄位格式
df_report.loc[:, "ReportDate"] = df_report.loc[:, "ReportDate"].apply(lambda s: "/".join([s.split("-")[2],s.split("-")[1],s.split("-")[0]]) 
                                                                         if isinstance(s, str) else "1/1/2000")


In [24]:
df_report.head()

,ReportDate,area,Serotype,outbreaks,RelatedReports,DateOfConfirmationOfTheEvent,ManifestationOfDisease,DateEventResolved,ReasonForNotification,ThisEventPertainsTo,...,DateOfPreviousOccurrence,DateSubmittedToOie,ReportType,lat,description,DateOfStartOfTheEvent,lon,_id,CausalAgent,url
0,06/08/2021,Chinese Taipei,H5N5,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,2021-08-06_Chinese Taipei,HPAI,https://wahis.oie.int/#/events?viewAll=true
1,05/08/2021,Togo,H5N1,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,2021-08-05_Togo,HPAI,https://wahis.oie.int/#/events?viewAll=true
2,05/08/2021,France,H5N8,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,2021-08-05_France,HPAI,https://wahis.oie.int/#/events?viewAll=true
3,04/08/2021,Lithuania,H7N7,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,2021-08-04_Lithuania,HPAI,https://wahis.oie.int/#/events?viewAll=true
4,04/08/2021,Lithuania,H5N8,2,None,None,None,None,None,None,...,None,None,None,None,None,None,None,2021-08-04_Lithuania,HPAI,https://wahis.oie.int/#/events?viewAll=true


# 儲存

In [25]:
df_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   ReportDate                    37 non-null     object
 1   area                          37 non-null     object
 2   Serotype                      37 non-null     object
 3   outbreaks                     37 non-null     int64 
 4   RelatedReports                0 non-null      object
 5   DateOfConfirmationOfTheEvent  0 non-null      object
 6   ManifestationOfDisease        0 non-null      object
 7   DateEventResolved             0 non-null      object
 8   ReasonForNotification         0 non-null      object
 9   ThisEventPertainsTo           0 non-null      object
 10  NatureOfDiagnosis             0 non-null      object
 11  DateOfPreviousOccurrence      0 non-null      object
 12  DateSubmittedToOie            0 non-null      object
 13  ReportType            